In [ ]:
import numpy as numpy
from matplotlib import pyplot as matplot
%matplotlib inline
import pandas as pandas
import math
from sklearn import preprocessing
from sklearn import svm
import cv2
import os
import ntpath
from sklearn.metrics.pairwise import cosine_similarity
import fnmatch

data = []
Filename = []
Person = []
Testdata = []
TestFilename = []
TestPerson = []


def crop_image(img,tol=0):
    mask = img>tol
    return img[numpy.ix_(mask.any(1),mask.any(0))]
def thinning(img):
    #img = cropimage
    size = numpy.size(img)
    skel = numpy.zeros(img.shape,numpy.uint8)

    ret,img = cv2.threshold(img,127,255,0)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))
    done = False

    while( not done):
        eroded = cv2.erode(img,element)
        temp = cv2.dilate(eroded,element)
        temp = cv2.subtract(img,temp)
        skel = cv2.bitwise_or(skel,temp)
        img = eroded.copy()

        zeros = size - cv2.countNonZero(img)
        if zeros==size:
            done = True

  #  plt.imshow(skel,'gray')
   # plt.show()
    return skel
def coords(thinnedimage):
    rows,cols=thinnedimage.shape
    img_topleft=thinnedimage[0:int(rows/2),0:int(cols/2)]
    img_topright=thinnedimage[int(rows/2)+1:rows,0:int(cols/2)]
    img_bottomleft=thinnedimage[0:int(rows/2),int(cols/2)+1:cols]
    img_bottomright=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:cols]
    
    topleft_x,topleft_y=COG(img_topleft)
    topright_x,topright_y=COG(img_topright)
    bottomleft_x,bottomleft_y=COG(img_bottomleft)
    bottomright_x,bottomright_y=COG(img_bottomright)

    return topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y

def COG(img):
    x_cor=0
    xrun_sum=0
    y_cor=0
    yrun_sum=0
    #print(img.shape)
    for i in range(img.shape[0]):
        x_cor+=sum(img[i])*i/255
        xrun_sum+=sum(img[i])/255

    for i in range(img.shape[1]):
        y_cor+=sum(img[:,i])*i/255
        yrun_sum+=sum(img[:,i])/255
        #print(img.shape[1]) 
        if yrun_sum==0:
            x_pos=0
        else:
            x_pos=y_cor/(yrun_sum)
        if xrun_sum==0:
            y_pos=0
        else:
            y_pos=x_cor/(xrun_sum)
        
   # print(x_pos)
  #  print(y_pos)
    
    return (x_pos/img.shape[1],y_pos/img.shape[0])

def tan_i(x):
    #print(x)
    if x[0]==0:
        return 90
    return math.degrees(math.atan(x[1]/x[0]))

def tan(thinnedimage):
    rows,cols=thinnedimage.shape
    
    img_tl1=thinnedimage[0:int(rows/2),0:int(cols/4)]
    img_tl2=thinnedimage[0:int(rows/2),int(cols/4)+1:int(cols/2)]
    
    img_tr1=thinnedimage[0:int(rows/2),int(cols/2)+1:int(0.75*cols)]
    img_tr2=thinnedimage[0:int(rows/2),int(0.75*cols)+1:cols]
    
    img_bl1=thinnedimage[int(rows/2)+1:rows,0:int(cols/4)]
    img_bl2=thinnedimage[int(rows/2)+1:rows,int(cols/4)+1:int(cols/2)]
    
    img_br1=thinnedimage[int(rows/2)+1:rows,int(cols/2)+1:int(0.75*cols)]
    img_br2=thinnedimage[int(rows/2)+1:rows,int(0.75*cols)+1:cols]
    

    #plt.imshow(timg,'gray')
    #plt.show()
    
    tl1=tan_i(COG(img_tl1))
    tl2=tan_i(COG(img_tl2))
    tr1=tan_i(COG(img_tr1))
    tr2=tan_i(COG(img_tr2))
    bl1=tan_i(COG(img_bl1))
    bl2=tan_i(COG(img_bl2))
    br1=tan_i(COG(img_br1))
    br2=tan_i(COG(img_br2))
    
    #plt.imshow(img_br1,'gray')
    #plt.show()
    #print(COG(img_br1))
    return tl1,tl2,tr1,tr2,bl1,bl2,br1,br2

def createfingerprint_Genuine(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    #Thinning the image!

    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
        #Extracting features

    a=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio"])
    data.append(a)

def createfingerprint_Test(image):
    aspect_ratio = 1.0*(image.shape[1]/image.shape[0])
    ret,thresh1 = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
    ret,thresh2 = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    ret,thresh3 = cv2.threshold(image,127,255,cv2.THRESH_TRUNC)
    ret,thresh4 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO)
    ret,thresh5 = cv2.threshold(image,127,255,cv2.THRESH_TOZERO_INV)
    blur = cv2.GaussianBlur(thresh1,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    image=numpy.invert(th3)
    
    cropimage=crop_image(image,tol=0)
    area=cv2.countNonZero(cropimage)
    area=cv2.countNonZero(cropimage)/(cropimage.shape[0]*cropimage.shape[1])

    img1=numpy.invert(cropimage)
    connectcomp=cv2.connectedComponents(img1)[0]

    thinnedimage=thinning(cropimage)
    #Thinning the image!

    topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y=coords(thinnedimage)
    tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y = tan(thinnedimage)
    
        #Extracting features

    b=pandas.Series([area,connectcomp,topleft_x,topleft_y,topright_x,topright_y,bottomleft_x,bottomleft_y,bottomright_x,bottomright_y,tan_topleft_x, tan_topleft_y, tan_topright_x, tan_topright_y, tan_bottomleft_x, tan_bottomleft_y, tan_bottomright_x, tan_bottomright_y,aspect_ratio],index=
                    ["Norm_area","connected_comps","topleft_x","topleft_y","topright_x","topright_y","bottomleft_x","bottomleft_y","bottomright_x","bottomright_y","tan_topleft_x", "tan_topleft_y", "tan_topright_x", "tan_topright_y", "tan_bottomleft_x", "tan_bottomleft_y", "tan_bottomright_x", "tan_bottomright_y","Aspect_Ratio"])
    Testdata.append(b)    
    

def create_training_Genuine_dataset(m):
    os.chdir(Genuinedir)
    filecount = len(fnmatch.filter(os.listdir(), '*.png'))
    n = int(filecount/m)
    for j in range(1,m+1):
        for i in range(1,n+1):
            path = f"D:\\Data Science\\Python\\Signature Verification\\Training Data\\Train_Genuine\\00{j}_{i}.png"
            fn = ntpath.basename(path)
            Filename.append(fn)
            tn = j
            Person.append(tn)
            image = cv2.imread(path,0)
            createfingerprint_Genuine(image)
            Original_Signature_Table=pandas.DataFrame(data)
            Original_Cosine = pandas.DataFrame(cosine_similarity(Original_Signature_Table), index = None)
            
    Original_Signature_Table['Filename'] = Filename
    Original_Cosine['Filename'] = Filename
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Person'] = Person
    Trainingdatacol = Original_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Original_Signature_Table = Original_Signature_Table[Trainingdatacol]
    Original_Signature_Table['Category'] = "Genuine"
    Cosinecolumns = Original_Cosine.columns.tolist()
    Cosinecolumns = Cosinecolumns[-1:]+Cosinecolumns[:-1]
    Original_Cosine = Original_Cosine[Cosinecolumns]
    #Original_Signature_Table = pandas.concat([Original_Signature_Table,Cosine],axis = 1)
    Original_Signature_Table.to_csv('Training.csv', index = False)
    Original_Cosine.to_csv('Cosine_Training.csv', index = False)
    Summary_Table = pandas.DataFrame(Original_Signature_Table.describe()).T
    Summary_Table.to_csv('Training_Summary.csv', index = False)
    Original_Signature_Table
    Original_Cosine
    print('Preprocessing of Training Data Completed')
    
def create_test_dataset(x):
    os.chdir(Testdir)
    Testcount = len(fnmatch.filter(os.listdir(), '*.png'))
    y = int(Testcount/x)
    for j in range(1,x+1):
        for i in range(1,y+1):
            path = f"D:\\Data Science\\Python\\Signature Verification\\Training Data\\Test\\00{j}_{i}.png"
            fn = ntpath.basename(path)
            tn = j
            TestFilename.append(fn)
            TestPerson.append(tn)
            image = cv2.imread(path,0)
            createfingerprint_Test(image)
            Test_Signature_Table=pandas.DataFrame(Testdata)
            Test_Cosine = pandas.DataFrame(cosine_similarity(Test_Signature_Table), index = None)
            
            

    Test_Signature_Table['Filename'] = TestFilename
    Test_Cosine['Filename'] = TestFilename
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Person'] = TestPerson
    Trainingdatacol = Test_Signature_Table.columns.tolist()
    Trainingdatacol = Trainingdatacol[-1:]+Trainingdatacol[:-1]
    Test_Signature_Table = Test_Signature_Table[Trainingdatacol]
    Test_Signature_Table['Category'] = "Test"
    Cosinecolumns = Test_Cosine.columns.tolist()
    Cosinecolumns = Cosinecolumns[-1:]+Cosinecolumns[:-1]
    Test_Cosine = Test_Cosine[Cosinecolumns]
    #Original_Signature_Table = pandas.concat([Original_Signature_Table,Cosine],axis = 1)
    Test_Signature_Table.to_csv('Test.csv', index = False)
    Test_Cosine.to_csv('Cosine_Test.csv', index = False)
    Summary_Table = pandas.DataFrame(Test_Signature_Table.describe()).T
    Summary_Table.to_csv('Test_Summary.csv', index = False)
    Test_Signature_Table
    Test_Cosine
    print('Preprocessing of Test Data Completed')

In [ ]:
Genuinedir = 'D:\Data Science\Python\Signature Verification\Training Data\Train_Genuine'
create_training_Genuine_dataset(10)

In [ ]:
Testdir = 'D:\Data Science\Python\Signature Verification\Training Data\Test'
create_test_dataset(2)

In [ ]:
os.chdir(Genuinedir)
Original_Signature_Table = pandas.read_csv("Training.csv")
Cosine_Training = pandas.read_csv("Cosine_Training.csv")
os.chdir(Testdir)
Test_Signature_Table = pandas.read_csv("Test.csv")
Cosine_Test = pandas.read_csv("Cosine_Test.csv")

In [ ]:
Original_Signature_Table

In [ ]:
Test_Signature_Table

In [ ]:
Original_Signature_Table.groupby('Person').describe().T

In [ ]:
Mean_Original_Signature = pandas.DataFrame(Original_Signature_Table.groupby('Person').mean())

In [ ]:
Mean_Test_Signature = pandas.DataFrame(Test_Signature_Table.groupby('Person').mean())